In [ ]:
!pip install -U langchain langchain-community gradio faiss-cpu huggingface_hub

In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload 'Training Dataset.csv'

In [ ]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "hf_your_token_here"  # Replace with your actual token

In [ ]:
with open("retriever.py", "w") as f:
    f.write('''from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter

def get_vectorstore(csv_path):
    loader = CSVLoader(file_path=csv_path)
    documents = loader.load()

    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    docs = text_splitter.split_documents(documents)

    embeddings = HuggingFaceEmbeddings()
    db = FAISS.from_documents(docs, embeddings)
    return db.as_retriever()''')

In [ ]:
with open("qa_chain.py", "w") as f:
    f.write('''import os
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_community.llms import HuggingFaceHub

class QABot:
    def __init__(self, retriever):
        self.retriever = retriever
        self.memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        self.llm = HuggingFaceHub(
            repo_id="google/flan-t5-base",
            model_kwargs={"temperature": 0.1, "max_length": 512},
            huggingfacehub_api_token=os.environ["HUGGINGFACEHUB_API_TOKEN"]
        )
        self.chain = ConversationalRetrievalChain.from_llm(
            llm=self.llm,
            retriever=self.retriever,
            memory=self.memory
        )
    def ask(self, query):
        return self.chain.run(query)''')

In [ ]:
from retriever import get_vectorstore
from qa_chain import QABot
import gradio as gr

retr = get_vectorstore("Training Dataset.csv")
bot = QABot(retriever=retr)

def chatbot_reply(message, history):
    try:
        return bot.ask(message)
    except Exception as e:
        return f"⚠️ Error: {str(e)}"

gr.ChatInterface(
    fn=chatbot_reply,
    title="📊 Loan Approval Chatbot",
    description="Ask anything about the loan dataset."
).launch()